In [1]:
required_packages <- c(
  "forecast",       # ARIMAX modeling
  "dplyr",          # Data manipulation
  "tidyr",          # pivot_wider/pivot_longer
  "prophet",        # xreg forecasting
  "thief",          # h
  "TSrepr"          # Errors
)

# Install missing packages
new_packages <- required_packages[!required_packages %in% installed.packages()[,"Package"]]
if(length(new_packages)) install.packages(new_packages)

# Load all packages
invisible(lapply(required_packages, library, character.only = TRUE))

also installing the dependencies ‘numDeriv’, ‘backports’, ‘abind’, ‘tensorA’, ‘distributional’, ‘ps’, ‘TTR’, ‘cpp11’, ‘checkmate’, ‘matrixStats’, ‘posterior’, ‘callr’, ‘processx’, ‘quadprog’, ‘quantmod’, ‘timechange’, ‘inline’, ‘gridExtra’, ‘loo’, ‘pkgbuild’, ‘QuickJSR’, ‘desc’, ‘SparseM’, ‘MatrixModels’, ‘survival’, ‘fracdiff’, ‘lmtest’, ‘nnet’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘dygraphs’, ‘extraDistr’, ‘lubridate’, ‘RcppParallel’, ‘rstan’, ‘rstantools’, ‘StanHeaders’, ‘xts’, ‘BH’, ‘RcppEigen’, ‘hts’, ‘quantreg’, ‘wavelets’, ‘dtt’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Rcpp

Loading required package:

In [2]:
calendar <- read.csv("calendar.csv", stringsAsFactors = FALSE)
sales <- read.csv("sales_train_validation.csv", stringsAsFactors = FALSE)
prices <- read.csv("sell_prices.csv", stringsAsFactors = FALSE)

In [3]:
calendar <- calendar %>%
  mutate(
    date = as.Date(date),
    is_holiday = 0  # Initialize all as 0 (not holidays)
  )

# Loop through each row (day) and check if it has any holiday-related events
for (i in 1:nrow(calendar)) {
  if (calendar$event_name_1[i] != "" | calendar$event_type_1[i] != "" |
      calendar$event_name_2[i] != "" | calendar$event_type_2[i] != "") {
    calendar$is_holiday[i] <- 1  # Set is_holiday to 1 if there's any event
  }
}

In [4]:
sales <- sales %>% mutate(row_id = row_number())
item_metadata <- sales %>% select(row_id, dept_id)
sales_long <- sales %>%
  select(row_id, starts_with("d_")) %>%
  pivot_longer(cols = starts_with("d_"), names_to = "day", values_to = "value")

In [5]:
dept_sales <- sales_long %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  left_join(item_metadata, by = "row_id") %>%
  group_by(date, dept_id) %>%
  summarise(dept_sales = sum(value, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(
    names_from = dept_id,
    values_from = dept_sales,
    values_fill = 0
  ) %>%
  arrange(date)

In [6]:
sales_with_categories <- sales %>%
  mutate(
    row_id = row_number(),
    category = case_when(
      grepl("^FOODS", item_id) ~ "FOODS",
      grepl("^HOBBIES", item_id) ~ "HOBBIES",
      grepl("^HOUSEHOLD", item_id) ~ "HOUSEHOLD",
      TRUE ~ "OTHER"
    )
  ) %>%
  filter(category != "OTHER")

In [7]:
category_sales <- sales_with_categories %>%
  select(row_id, category, starts_with("d_")) %>%
  pivot_longer(cols = starts_with("d_"), names_to = "day", values_to = "value")

In [8]:
category_sales_aggregated <- category_sales %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  group_by(date, category) %>%
  summarise(category_sales = sum(value, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(
    names_from = category,
    values_from = category_sales,
    values_fill = 0
  ) %>%
  arrange(date)

In [9]:
# Main function for univariate forecasting
forecastSingleModel <- function(xx,steps.ahead,model) {
  try(yy <- switch(model,                  
                   "strend" = forecast::forecast(forecast::tslm(xx ~ trend + season),level=c(80),h=steps.ahead),
                   "snaive" = forecast::snaive(xx,h=steps.ahead,level=c(80)),                   
                   "croston" = forecastCroston(xx,steps.ahead),
                   "holtw" = forecast::forecast(HoltWinters(xx),level=c(80),h=steps.ahead),
                   "theta" = forecast::thetaf(xx,h=steps.ahead,level=c(80)),
                   "thetah" = thief::thief(xx,comb="mse",usemodel="theta",h=steps.ahead),
                   "ets" = forecast::forecast(forecast::ets(xx),level=c(80),h=steps.ahead),
                   "etsd" = forecast::forecast(forecast::stlm(xx,method="ets",robust=T,s.window="per"),level=c(80),h=steps.ahead),
                   "etsh" = thief::thief(xx,comb="mse",usemodel="ets",h=steps.ahead),
                   "etsb" = forecast::forecast(forecast::baggedETS(xx),level=c(80),h=steps.ahead),
                   "arima" = forecast::forecast(forecast::auto.arima(xx,seasonal=T),level=c(80),h=steps.ahead),
                   "arimad" = forecast::forecast(forecast::stlm(xx,method="arima",robust=T,s.window="per"),level=c(80),h=steps.ahead),
                   "arimah" = thief::thief(xx,comb="mse",usemodel="arima",h=steps.ahead),
                   "tbats" = forecast::forecast(forecast::tbats(xx),level=c(80),h=steps.ahead),
                   "prophet" = forecastProphet(xx,h=steps.ahead,myfreq="month"),
                   forecast::naive(xx,h=steps.ahead,level=c(80))), silent=T)
  return(yy)
}

In [10]:
# Wrapper around Croston forecasting model
forecastCroston <- function(xx,fh) {
  const <- ifelse(any(xx<0),min(xx),0)
  ts_pos <- xx + const
  yy <- forecast::croston(ts_pos,h=fh)
  yy$mean <- yy$mean - const
  return(yy)
}

In [11]:
# Function to decide if a time series is seasonal
SeasonalityTest <- function(input, ppy) {
  tcrit <- 1.645
  if (length(input)<3*ppy){
    test_seasonal <- F
  } else {
    xacf <- acf(input, plot = F)$acf[-1, 1, 1]
    clim <- tcrit/sqrt(length(input)) * sqrt(cumsum(c(1, 2*xacf^2)))
    test_seasonal <- ( abs(xacf[ppy]) > clim[ppy] )
    if (is.na(test_seasonal)==T) test_seasonal <- F
  }
  return(test_seasonal)
}

In [12]:
# Wrapper around Prophet forecasting model
forecastProphet <- function(ts_temp=NULL,level=80,h=1,fan=FALSE,myfreq="month") {
  ts_obs <- seq(as.Date("2011/01/29"), by = myfreq, length.out = length(ts_temp))
  tseries <- data.frame(ds=ts_obs,y=ts_temp)
  if (fan) level <- seq(from=51,to=99,by=3)
  fc_temp <- forecast::meanf(ts_temp,h=h,level=level)
  fc_temp$method <- "Prophet"
  fc_temp$level <- level  
  for (i in seq(1,length(level))) {
    log <- utils::capture.output(my_model <- prophet::prophet(tseries,interval.width=level[i]/100))
    future <- prophet::make_future_dataframe(my_model, periods=h, freq=myfreq)
    fc_prophet <- predict(my_model,future)
    myidx <- seq(length(fc_prophet$yhat)-h+1,length(fc_prophet$yhat))
    fc_temp$mean[1:length(fc_temp$mean)] <- fc_prophet$yhat[myidx]
    fc_temp$fitted[1:length(fc_temp$fitted)] <- fc_prophet$yhat[1:(length(fc_prophet$yhat)-h)]
    if (length(level) > 1) {
      fc_temp$lower[,i] <- fc_prophet$yhat_lower[myidx]
      fc_temp$upper[,i] <- fc_prophet$yhat_upper[myidx]
    } else {
      fc_temp$lower <- fc_prophet$yhat_lower[myidx]
      fc_temp$upper <- fc_prophet$yhat_upper[myidx]
    }
  }
  return(fc_temp)
}

In [13]:
# Function to evaluate forecasting models and schemes
evaluateModels <- function(xx,fh,modelsList,winCount=3,winOverlap=0.5) {
  # extract windows
  seriesList <- NULL
  realValues <- NULL
  windowLast <- (length(xx)-fh+1):length(xx)
  for (i in 1:winCount) {
    idx <- windowLast - (i-1)*floor((1-winOverlap)*fh)
    trainSeries <- stats::window(xx, start=head(as.numeric(time(xx)),1),
                                 end=head(as.numeric(time(xx))[idx-1],1) )
    seriesList <- c(seriesList,list(trainSeries))
    realValues <- c(realValues,xx[idx])
  }
  # validate main models
  stackedErrors <- NULL
  inSampleMAE <- mean(abs(diff(xx[1:(tail(idx,1)-1)])))
  for (model in modelsList) {
    foreValues <- NULL
    start_time <- Sys.time()
    for (trainSeries in seriesList) {
      yy <- forecastSingleModel(trainSeries,fh,model)
      foreValues <- c(foreValues,as.numeric(yy$mean))
    }
    end_time <- Sys.time()
    runTime <- c(elapsed=as.numeric(difftime(end_time,start_time,units="secs")))
    foreErrors <- forecastingErrors(realValues,foreValues,inSampleMAE)
    modelPerformance <- data.frame(t(runTime),foreErrors,stringsAsFactors=F)
    stackedErrors <- rbind(stackedErrors,modelPerformance)
  }
  stackedErrors$elapsed  <- stackedErrors$elapsed/winCount
  stackedErrors$elapsed[stackedErrors$elapsed==0] <- .Machine$double.eps
  idx <- sort.list(stackedErrors$elapsed)
  stackedErrors <- stackedErrors[idx, ]
  modelsList <- modelsList[idx]
  rankedErrors <- sapply(stackedErrors, function(x) rank(x,ties.method="first"))
  if (is.matrix(rankedErrors)) {
    averaged <- rowMeans(rankedErrors[,-1])
    final <- rank(averaged,ties.method="first")
  } else {
    final <- 1
  }
  modelsTable <- data.frame(rank=final,model=modelsList,stackedErrors,stringsAsFactors=F)
  modelsTable <- modelsTable[sort.list(modelsTable$rank), ]
  return(modelsTable)
}

In [14]:
# Helper function to estimate various forecasting errors
forecastingErrors <- function(realVals,foreVals,inSamp) {
  RMSE <- TSrepr::rmse(realVals,foreVals)
  MDAE <- TSrepr::mdae(realVals,foreVals)
  MASE <- TSrepr::mae(realVals,foreVals) / inSamp
  SMAPE <- TSrepr::smape(realVals,foreVals)
  MAAPE <- TSrepr::maape(realVals,foreVals)
  data.frame(RMSE,MDAE,MASE,SMAPE,MAAPE)
}

In [15]:
model.names <- c("strend", "snaive", "croston", "holtw", "theta", "ets", "etsd", "arima", "arimad", "tbats", "prophet", "thetah", "etsh", "arimah", "etsb")
steps.ahead <- 28
windows.num <- 3

In [16]:
xx1 <- ts(dept_sales$FOODS_1, start=as.numeric(format(dept_sales$date[1], "%Y")), frequency=365)
xx2 <- ts(dept_sales$FOODS_2, start=as.numeric(format(dept_sales$date[1], "%Y")), frequency=365)
xx3 <- ts(dept_sales$FOODS_3, start=as.numeric(format(dept_sales$date[1], "%Y")), frequency=365)

xx4 <- ts(dept_sales$HOBBIES_1, start=as.numeric(format(dept_sales$date[1], "%Y")), frequency=365)
xx5 <- ts(dept_sales$HOBBIES_2, start=as.numeric(format(dept_sales$date[1], "%Y")), frequency=365)

xx6 <- ts(dept_sales$HOUSEHOLD_1, start=as.numeric(format(dept_sales$date[1], "%Y")), frequency=365)
xx7 <- ts(dept_sales$HOUSEHOLD_2, start=as.numeric(format(dept_sales$date[1], "%Y")), frequency=365)

xx_foods <- ts(category_sales_aggregated$FOODS, 
               start = as.numeric(format(category_sales_aggregated$date[1], "%Y")), 
               frequency = 365)
xx_hobbies <- ts(category_sales_aggregated$HOBBIES, 
                 start = as.numeric(format(category_sales_aggregated$date[1], "%Y")), 
                 frequency = 365)
xx_household <- ts(category_sales_aggregated$HOUSEHOLD, 
                   start = as.numeric(format(category_sales_aggregated$date[1], "%Y")), 
                   frequency = 365)

In [17]:
prognozuotiIrIssaugoti <- function(ts_data, pavadinimas, steps_ahead = 28) {
  # Įvertinti modelius
  rezultatai <- evaluateModels(ts_data, steps_ahead, model.names, windows.num)
  
  # Pasirinkti geriausią modelį (pagal mažiausią rank reikšmę)
  geriausias_modelis <- rezultatai$model[1]
  
  # Atlikti prognozę su geriausiu modeliu
  prognoze <- forecastSingleModel(ts_data, steps_ahead, geriausias_modelis)
  
  # Sukurti rezultatų data.frame
  rezultatu_df <- data.frame(
    data = seq.Date(
      from = as.Date(max(dept_sales$date)) + 1,
      by = "day",
      length.out = steps_ahead
    ),
    kintamasis = pavadinimas,
    prognoze = as.numeric(prognoze$mean),
    modelis = geriausias_modelis,
    stringsAsFactors = FALSE
  )
  
  # Grąžinti rezultatus
  return(rezultatu_df)
}

In [18]:
visos_prognozes <- list()

# Departamentai
visos_prognozes$FOODS_1 <- prognozuotiIrIssaugoti(xx1, "FOODS_1")
visos_prognozes$FOODS_2 <- prognozuotiIrIssaugoti(xx2, "FOODS_2")
visos_prognozes$FOODS_3 <- prognozuotiIrIssaugoti(xx3, "FOODS_3")
visos_prognozes$HOBBIES_1 <- prognozuotiIrIssaugoti(xx4, "HOBBIES_1")
visos_prognozes$HOBBIES_2 <- prognozuotiIrIssaugoti(xx5, "HOBBIES_2")
visos_prognozes$HOUSEHOLD_1 <- prognozuotiIrIssaugoti(xx6, "HOUSEHOLD_1")
visos_prognozes$HOUSEHOLD_2 <- prognozuotiIrIssaugoti(xx7, "HOUSEHOLD_2")

# Kategorijos
visos_prognozes$FOODS <- prognozuotiIrIssaugoti(xx_foods, "FOODS")
visos_prognozes$HOBBIES <- prognozuotiIrIssaugoti(xx_hobbies, "HOBBIES")
visos_prognozes$HOUSEHOLD <- prognozuotiIrIssaugoti(xx_household, "HOUSEHOLD")

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in HoltWinters(xx):
“optimization difficulties: ERROR: ABNORMAL_TERMINATION_IN_LNSRCH”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::

In [19]:
state_sales <- sales %>%
  mutate(row_id = row_number()) %>%
  select(row_id, state_id, starts_with("d_")) %>%
  pivot_longer(cols = starts_with("d_"), names_to = "day", values_to = "value")

state_sales_aggregated <- state_sales %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  group_by(date, state_id) %>%
  summarise(state_sales = sum(value, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(
    names_from = state_id,
    values_from = state_sales,
    values_fill = 0
  ) %>%
  arrange(date)

# Laiko eilutės
xx_CA <- ts(state_sales_aggregated$CA, start = as.numeric(format(state_sales_aggregated$date[1], "%Y")), frequency = 365)
xx_TX <- ts(state_sales_aggregated$TX, start = as.numeric(format(state_sales_aggregated$date[1], "%Y")), frequency = 365)
xx_WI <- ts(state_sales_aggregated$WI, start = as.numeric(format(state_sales_aggregated$date[1], "%Y")), frequency = 365)

visos_prognozes$CA <- prognozuotiIrIssaugoti(xx_CA, "CA")
visos_prognozes$TX <- prognozuotiIrIssaugoti(xx_TX, "TX")
visos_prognozes$WI <- prognozuotiIrIssaugoti(xx_WI, "WI")

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

In [23]:
store_sales <- sales %>%
  mutate(row_id = row_number()) %>%
  select(row_id, store_id, starts_with("d_")) %>%
  pivot_longer(cols = starts_with("d_"), names_to = "day", values_to = "value")

store_sales_aggregated <- store_sales %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  group_by(date, store_id) %>%
  summarise(store_sales = sum(value, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(
    names_from = store_id,
    values_from = store_sales,
    values_fill = 0
  ) %>%
  arrange(date)

# Laiko eilutės (10 parduotuvių)
for (sid in names(store_sales_aggregated)[-1]) {
  ts_obj <- ts(store_sales_aggregated[[sid]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
  visos_prognozes[[sid]] <- prognozuotiIrIssaugoti(ts_obj, sid)
}

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

In [41]:
sales_with_categories <- sales %>%
  mutate(
    row_id = row_number(),
    category = case_when(
      grepl("^FOODS", item_id) ~ "FOODS",
      grepl("^HOBBIES", item_id) ~ "HOBBIES",
      grepl("^HOUSEHOLD", item_id) ~ "HOUSEHOLD",
      TRUE ~ "OTHER"
    )
  ) %>%
  filter(category != "OTHER")

# Sukuriam state × category laiko eilutes
state_category_sales <- sales_with_categories %>%
  select(row_id, state_id, category, starts_with("d_")) %>%
  pivot_longer(cols = starts_with("d_"), names_to = "day", values_to = "value") %>%
  left_join(calendar %>% select(day = d, date), by = "day") %>%
  filter(!is.na(state_id), !is.na(category)) %>%
  group_by(date, state_id, category) %>%
  summarise(sc_sales = sum(value, na.rm = TRUE), .groups = "drop") %>%
  mutate(state_category = paste(state_id, category, sep = "_")) %>%
  select(date, state_category, sc_sales) %>%
  pivot_wider(names_from = state_category, values_from = sc_sales, values_fill = 0) %>%
  arrange(date)

# Prognozės kiekvienai valstijos–kategorijos kombinacijai (9 laiko eilutės)
for (id in names(state_category_sales)[-1]) {
  ts_obj <- ts(state_category_sales[[id]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
  visos_prognozes[[id]] <- prognozuotiIrIssaugoti(ts_obj, id)
}

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

In [26]:
# Pridedame kategoriją prie kainų lentelės
prices_cat <- prices %>%
  left_join(sales %>% select(item_id, cat_id) %>% distinct(), by = "item_id")

daily_avg_prices <- prices_cat %>%
  left_join(calendar %>% select(date, wm_yr_wk), by = "wm_yr_wk") %>%
  group_by(date, cat_id) %>%
  summarise(avg_price = mean(sell_price, na.rm = TRUE), .groups = "drop") %>%
  complete(date, cat_id) %>%  # įterpia trūkstamas kombinacijas
  arrange(cat_id, date) %>%
  group_by(cat_id) %>%
  mutate(ma_7 = ifelse(row_number() >= 7, zoo::rollmean(avg_price, k = 7, fill = NA, align = "right"), NA)) %>%
  ungroup() %>%
  select(date, cat_id, ma_7) %>%
  pivot_wider(names_from = cat_id, values_from = ma_7) %>%
  arrange(date)

Warning message in left_join(., calendar %>% select(date, wm_yr_wk), by = "wm_yr_wk"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 897 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [50]:
for (col in names(daily_avg_prices)[-1]) {
  series_raw <- daily_avg_prices[[col]]

  # 1. Pašaliname NA
  series_clean <- series_raw[!is.na(series_raw)]

  # 2. Patikriname ar pakanka duomenų
  if (length(series_clean) < 10 || sum(series_clean) == 0) {
    message(paste("⚠️ Praleidžiama per trumpa arba tuščia serija:", col))
    next
  }

  # 3. Sukuriame laiko eilutę
  ts_obj <- ts(series_clean, frequency = 365)

  # 4. Bandome prognozuoti
  rezultat <- try(prognozuotiIrIssaugoti(ts_obj, col), silent = TRUE)

  if (!inherits(rezultat, "try-error") && !is.null(rezultat)) {
    visos_prognozes[[col]] <- rezultat
  } else {
    message(paste("❌ Nepavyko prognozuoti serijai:", col))
  }
}

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

In [51]:
galutines_prognozes <- do.call(rbind, visos_prognozes)
rownames(galutines_prognozes) <- NULL

In [52]:
write.csv(galutines_prognozes, "isplestos_prognozes.csv", row.names = FALSE)

In [53]:
prognozes_platus <- galutines_prognozes %>%
  select(data, kintamasis, prognoze) %>%
  pivot_wider(names_from = kintamasis, values_from = prognoze)

prognozes_platus <- prognozes_platus %>%
  left_join(calendar %>% select(date, is_holiday), by = c("data" = "date"))

In [54]:
write.csv(prognozes_platus, "prognozes_isplestos_platus.csv", row.names = FALSE)

In [ ]:
---

In [29]:
output1 <- evaluateModels(xx1, steps.ahead, model.names, windows.num)
output1

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in HoltWinters(xx):
“optimization difficulties: ERROR: ABNORMAL_TERMINATION_IN_LNSRCH”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
11,1,prophet,4.317610582,632.2122,462.1735,1.446316,15.46715,15.74749
10,2,tbats,8.353650093,644.7020,370.1253,1.473571,15.56416,15.85515
15,3,etsb,8.032515446,655.5496,485.6869,1.497741,15.97551,16.44991
3,4,croston,51.297918161,676.9968,482.2968,1.548995,16.47242,17.05723
14,5,arimah,26.812866926,679.5355,436.1471,1.585083,16.65238,17.26468
9,6,arimad,0.474772612,695.7362,421.9791,1.613374,16.82689,17.46768
8,7,arima,5.562780539,729.2146,441.9302,1.705694,17.72160,18.48541
7,8,etsd,0.093582869,744.2779,552.7662,1.733619,18.34116,18.78613
1,9,strend,0.092392683,752.0001,515.6587,1.779573,18.84591,19.49570


In [30]:
output2 <- evaluateModels(xx2, steps.ahead, model.names, windows.num)
output2

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9,1,arimad,0.431502104,953.4693,473.9584,1.248841,11.94886,11.35679
4,2,holtw,0.047835827,1060.6552,691.1554,1.439867,13.98833,13.34120
7,3,etsd,0.144307295,1111.7226,656.4184,1.492758,14.37849,13.78789
14,4,arimah,35.712910414,1085.9317,647.6907,1.507349,14.42490,14.36684
11,5,prophet,4.060083071,1176.6712,673.8293,1.632696,15.67890,15.26442
10,6,tbats,9.960392316,1150.4841,809.5692,1.625291,15.68950,14.73342
12,7,thetah,0.057022492,1243.4210,737.0490,1.730811,16.91619,15.88930
3,8,croston,51.169277191,1236.1446,764.7052,1.713432,16.43764,16.14700
5,9,theta,0.024680853,1238.9449,767.6750,1.738762,16.96176,16.02047


In [31]:
output3 <- evaluateModels(xx3, steps.ahead, model.names, windows.num)
output3

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,27.371871154,2298.325,1475.656,0.8147363,9.094938,8.975712
10,2,tbats,16.161750793,2199.721,1680.286,0.8060101,9.131567,9.335833
8,3,arima,7.015057246,2390.817,1602.253,0.8627289,9.676219,9.624225
15,4,etsb,13.656520128,3720.098,1870.351,1.2288683,13.834319,12.594894
9,5,arimad,0.403464953,3244.336,2677.131,1.2509634,13.969655,13.978501
3,6,croston,51.126667658,3313.446,2628.958,1.2623926,14.216694,14.214304
4,7,holtw,0.048928261,3546.862,2361.544,1.2846752,14.259425,15.045573
11,8,prophet,4.192949533,3989.645,1880.828,1.3043860,14.774244,13.079839
1,9,strend,0.104709784,4199.443,2826.547,1.5696064,17.886383,16.854594


In [32]:
output4 <- evaluateModels(xx4, steps.ahead, model.names, windows.num)
output4

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,13.524810791,446.2608,313.9106,0.8887366,10.15642,10.02580
8,2,arima,5.983819087,460.5877,308.7564,0.9026498,10.31446,10.10952
10,3,tbats,11.751227140,444.3062,408.5701,0.9277214,10.66026,10.82664
4,4,holtw,0.048775832,565.5480,361.8368,1.0943731,12.50723,12.58246
9,5,arimad,0.483356396,535.2807,424.7890,1.1040127,12.63447,12.87836
13,6,etsh,0.236405532,532.9183,434.3314,1.1251715,12.94075,12.88216
6,7,ets,0.079008102,533.8936,432.1189,1.1268802,12.95984,12.90477
12,8,thetah,0.065984964,533.0396,441.7127,1.1307099,13.00489,12.97806
2,9,snaive,0.001317183,609.8529,363.5000,1.1527812,13.51692,12.84250


In [33]:
output5 <- evaluateModels(xx5, steps.ahead, model.names, windows.num)
output5

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
15,1,etsb,5.624564330,50.40777,35.51576,0.9155093,11.12980,11.39647
3,2,croston,51.680918932,51.37239,38.00000,0.9287998,11.29944,11.13101
13,3,etsh,0.218322198,51.20644,38.19870,0.9303816,11.30917,11.47818
8,4,arima,5.967562040,52.00627,38.78591,0.9467081,11.51499,11.38489
14,5,arimah,25.328696092,52.36446,39.97313,0.9440207,11.48473,11.50669
6,6,ets,0.056701819,52.54498,38.16303,0.9570389,11.63096,11.78752
10,7,tbats,6.803731600,55.09658,40.43672,0.9952168,12.12823,11.76369
1,8,strend,0.092059453,60.62894,38.74329,1.0870083,13.04995,13.89408
11,9,prophet,4.482448896,60.07667,42.12668,1.0809990,12.98069,13.94605


In [34]:
output6 <- evaluateModels(xx6, steps.ahead, model.names, windows.num)
output6

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8,1,arima,5.918062766,1031.175,629.4329,0.9580889,10.17478,10.11340
14,2,arimah,7.456800461,1018.374,682.5693,0.9662916,10.27711,10.30010
10,3,tbats,9.691789309,1022.272,775.4558,0.9766035,10.43872,10.61569
9,4,arimad,0.457408508,1102.033,884.0618,1.0605277,11.22836,11.24634
4,5,holtw,0.046489557,1430.858,1112.6169,1.3570463,14.48460,14.50003
2,6,snaive,0.001348178,1657.599,865.0000,1.4545554,16.00869,14.57487
11,7,prophet,4.274899562,1474.653,1335.9935,1.4798567,15.86239,16.04708
3,8,croston,50.971930742,1509.004,1307.5355,1.5246642,16.31360,16.65207
15,9,etsb,6.331165155,1547.126,1367.0096,1.6140787,17.24305,17.91032


In [35]:
output7 <- evaluateModels(xx7, steps.ahead, model.names, windows.num)
output7

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,10.175385157,227.4037,118.3483,0.7617345,7.957183,7.752851
8,2,arima,5.946825027,231.4417,125.1223,0.7912720,8.277022,8.134774
14,3,arimah,14.462505023,233.0019,139.7318,0.8076383,8.456508,8.368446
9,4,arimad,0.440443039,260.7025,148.8632,0.8915739,9.366860,9.275806
4,5,holtw,0.047938824,357.8740,201.8230,1.2621472,13.334423,13.293849
2,6,snaive,0.001952807,373.3395,239.5000,1.3590970,14.806521,14.062300
11,7,prophet,4.153954824,395.4144,239.3181,1.3499699,14.394827,13.397434
15,8,etsb,8.925388813,381.0127,277.9145,1.4446111,15.474164,15.179312
13,9,etsh,0.223790646,379.5674,307.2082,1.4910705,15.982705,16.026272


In [36]:
output8 <- evaluateModels(xx_foods, steps.ahead, model.names, windows.num)
output8

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,98.473170996,3411.108,2346.933,0.9292883,9.551559,9.575031
8,2,arima,6.584921519,3801.855,2445.002,1.0365197,10.682310,10.632123
10,3,tbats,15.538659811,4175.107,2566.998,1.0812418,11.482281,10.609222
4,4,holtw,0.044253111,4706.243,2752.919,1.2504857,12.776672,13.187209
9,5,arimad,0.472451766,4557.103,3263.718,1.2896483,13.160710,13.093710
15,6,etsb,12.570254008,5173.657,2868.687,1.3382651,13.812824,12.960904
11,7,prophet,4.245324055,5279.416,2804.409,1.3566960,14.017906,12.890918
3,8,croston,50.444922288,4785.045,3409.500,1.3365030,13.770672,13.802471
1,9,strend,0.090689421,5914.028,3111.280,1.5811156,16.531537,15.279363


In [37]:
output9 <- evaluateModels(xx_hobbies, steps.ahead, model.names, windows.num)
output9

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,25.788740953,473.2343,339.1572,0.9006540,9.729184,9.586347
8,2,arima,5.688719908,490.0359,314.6958,0.9129411,9.859933,9.629376
10,3,tbats,11.332127651,461.8393,414.6673,0.9148234,9.935731,9.995373
4,4,holtw,0.048113505,606.7895,369.9758,1.1186453,12.110781,12.159627
9,5,arimad,0.482876221,569.7910,397.3968,1.1199845,12.115162,12.294728
13,6,etsh,0.247347752,564.0372,456.4466,1.1461825,12.463580,12.369343
6,7,ets,0.069467068,565.2429,455.0378,1.1482330,12.485388,12.394940
12,8,thetah,0.047925949,563.9722,469.9265,1.1529002,12.537370,12.478298
2,9,snaive,0.001517614,650.7557,426.0000,1.1582663,12.790057,12.254970


In [38]:
output10 <- evaluateModels(xx_household, steps.ahead, model.names, windows.num)
output10

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,9.162416855,1138.355,834.3528,0.8786139,9.162848,9.184788
8,2,arima,5.926614364,1212.740,734.5047,0.8945098,9.282576,9.203353
14,3,arimah,7.218022426,1207.269,822.1092,0.9138435,9.508291,9.504978
9,4,arimad,0.439769506,1302.394,948.0087,0.9952806,10.311926,10.307344
4,5,holtw,0.048128366,1757.216,1016.7022,1.2904365,13.485401,13.309807
11,6,prophet,4.153370460,1824.394,1592.7273,1.4609083,15.383914,15.273611
2,7,snaive,0.001337369,2001.795,1081.0000,1.4301977,15.408858,14.144509
12,8,thetah,0.045539300,1836.354,1707.1734,1.4917309,15.707094,15.793158
5,9,theta,0.009436766,1837.836,1696.8264,1.4961596,15.753818,15.861365


In [39]:
output11 <- evaluateModels(xx_CA, steps.ahead, model.names, windows.num)
output11

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,18.504794121,1844.812,1340.939,0.7776783,8.274337,8.442896
9,2,arimad,0.432713509,1996.090,1241.812,0.8039133,8.389849,8.461885
8,3,arima,5.953370015,1951.622,1292.082,0.8122000,8.575890,8.543953
14,4,arimah,17.141201735,2011.192,1331.362,0.8410191,8.885506,8.832174
4,5,holtw,0.048018853,3121.458,2008.520,1.3008178,13.651851,14.150724
11,6,prophet,4.206032356,3211.451,1984.490,1.3076579,13.961267,13.336969
15,7,etsb,12.583965222,3177.216,2068.093,1.3008463,13.885093,13.366038
6,8,ets,0.113586823,3167.004,2127.818,1.3166657,14.064671,13.617718
13,9,etsh,0.296707948,3142.235,2380.838,1.3458948,14.392542,14.143825


In [40]:
output12 <- evaluateModels(xx_TX, steps.ahead, model.names, windows.num)
output12

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,38.977765481,1095.712,866.5305,0.6722270,8.093645,8.190588
10,2,tbats,17.332704544,1136.817,750.0091,0.6789439,8.220263,8.556109
8,3,arima,5.776325941,1137.201,888.3224,0.6898550,8.305069,8.341613
11,4,prophet,4.246768236,1589.345,997.9271,0.9401520,11.258126,10.828576
15,5,etsb,14.126809994,1561.017,1018.5812,0.9493101,11.370930,11.089434
9,6,arimad,0.463098685,1511.708,1271.6889,0.9481151,11.388084,11.475666
6,7,ets,0.125256459,1537.349,1056.0550,0.9784835,11.725166,11.694896
13,8,etsh,0.302102009,1536.360,1078.3477,0.9838593,11.789522,11.801961
3,9,croston,50.780981859,1551.938,1211.2848,1.0111911,12.112161,12.313898


In [41]:
output13 <- evaluateModels(xx_WI, steps.ahead, model.names, windows.num)
output13

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9,1,arimad,1.913261e+00,2143.789,1490.319,1.200115,12.98003,12.59352
14,2,arimah,1.853962e+02,2207.454,1585.133,1.270909,13.81285,13.53253
4,3,holtw,4.613105e-02,2338.185,1617.188,1.322143,14.39587,14.23433
8,4,arima,7.897079e+00,2537.541,1854.473,1.436582,15.63327,15.29200
3,5,croston,5.106820e+01,2666.764,2005.905,1.557037,16.96052,17.12566
11,6,prophet,4.236832e+00,2818.021,1970.534,1.566502,17.10629,15.99114
13,7,etsh,2.896183e-01,2681.424,2009.466,1.564464,17.02956,17.29657
1,8,strend,9.416533e-02,3041.647,1785.692,1.652320,18.18151,16.59152
7,9,etsd,1.370291e-01,2765.449,2038.333,1.632473,17.82589,17.40507


In [42]:
ts_obj <- ts(store_sales_aggregated[[2]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output14 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output14

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,9.52918514,621.1415,447.5132,0.7632370,10.52933,10.41059
8,2,arima,5.94612559,635.3931,450.7208,0.7786982,10.74504,10.61129
14,3,arimah,51.67452137,642.0017,435.3516,0.7854249,10.83429,10.68777
9,4,arimad,0.47049236,697.6666,601.0817,0.8957923,12.46490,12.72911
2,5,snaive,0.00136137,915.1336,474.0000,1.0623607,14.98112,14.46656
4,6,holtw,0.05151661,920.1484,571.4694,1.1121364,15.43796,16.19408
15,7,etsb,9.19463150,986.2600,575.4307,1.2112105,16.96058,16.04816
11,8,prophet,4.03567926,978.3882,695.5172,1.2555967,17.62038,17.09259
3,9,croston,50.44069902,961.5952,802.4032,1.2758475,17.91488,17.85086


In [43]:
ts_obj <- ts(store_sales_aggregated[[3]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output15 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output15

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in HoltWinters(xx):
“optimization difficulties: ERROR: ABNORMAL_TERMINATION_IN_LNSRCH”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8,1,arima,6.009766897,872.4650,415.2019,1.151997,14.15776,14.36938
10,2,tbats,19.464294036,861.2532,434.4788,1.133922,13.66199,14.46979
14,3,arimah,22.287179391,897.8135,404.6138,1.201524,14.68380,14.93003
9,4,arimad,0.436632077,893.2349,418.3684,1.201859,14.71878,15.13047
4,5,holtw,0.054346323,1138.7001,722.9896,1.773354,20.85877,21.48324
5,6,theta,0.013814052,1158.7309,771.9620,1.866128,22.25039,21.99203
12,7,thetah,0.044038852,1159.7862,777.5916,1.868355,22.27645,22.02368
1,8,strend,0.090104818,1572.9008,247.5055,1.918510,23.24557,17.92334
6,9,ets,0.121950308,1166.2700,823.1647,1.890312,22.52948,22.42516


In [44]:
ts_obj <- ts(store_sales_aggregated[[4]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output16 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output16

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,8.540021e+00,703.1252,550.0241,0.8253239,8.996926,9.294176
8,2,arima,6.639745e+00,772.5723,679.0755,0.9313593,10.080522,10.239817
14,3,arimah,1.472221e+02,760.8865,644.7095,0.9574534,10.357105,10.551855
9,4,arimad,4.351167e-01,862.5430,744.3469,1.0626928,11.417697,11.677312
2,5,snaive,1.431068e-03,1107.9840,524.0000,1.2055172,12.772978,12.516070
11,6,prophet,3.951607e+00,1046.0862,674.3035,1.1934907,12.920122,12.358217
15,7,etsb,1.052934e+01,1026.5907,748.2941,1.1884392,12.859482,12.486566
13,8,etsh,2.340391e-01,1016.9160,892.0825,1.2385578,13.414838,13.467113
6,9,ets,6.670507e-02,1018.4298,920.0170,1.2455494,13.490253,13.577548


In [45]:
ts_obj <- ts(store_sales_aggregated[[5]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output17 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output17

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,13.208378235,261.6534,138.9695,0.8515094,7.393887,7.149758
8,2,arima,5.713634253,264.2208,137.5769,0.8617813,7.506575,7.257658
10,3,tbats,6.967191060,286.5973,155.9831,0.9559430,8.344358,8.061583
5,4,theta,0.015036742,312.5604,175.4441,1.0425956,9.116935,8.789741
3,5,croston,49.832344611,308.4843,178.8882,1.0423681,9.112907,8.866688
12,6,thetah,0.044869582,313.8675,171.4392,1.0450488,9.139654,8.795785
4,7,holtw,0.053136746,309.8065,198.3832,1.0392122,9.004718,8.887782
6,8,ets,0.094097455,315.0861,171.1732,1.0473579,9.160544,8.805643
13,9,etsh,0.253120104,316.2679,171.2716,1.0491764,9.177440,8.807878


In [46]:
ts_obj <- ts(store_sales_aggregated[[6]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output18 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output18

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,9.799096266,398.4386,220.1718,0.6948674,9.275151,9.235252
14,2,arimah,6.975862900,418.1119,306.9018,0.7509617,9.858048,9.712865
8,3,arima,5.663654725,423.2743,292.7364,0.7600983,10.019167,9.865978
15,4,etsb,11.443131367,565.3454,327.7214,0.9758870,12.792010,12.052092
11,5,prophet,4.248567104,562.9878,361.5387,0.9859477,12.935859,12.227553
6,6,ets,0.081485589,551.1652,358.1357,0.9872981,12.953546,12.450675
13,7,etsh,0.243872643,550.0284,362.9295,0.9869756,12.950817,12.450960
3,8,croston,49.934359868,542.4997,402.2207,1.0340471,13.582384,13.481165
4,9,holtw,0.049617926,665.6545,351.4416,1.1400440,14.694147,15.012276


In [47]:
ts_obj <- ts(store_sales_aggregated[[7]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output19 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output19

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,11.455672026,456.5246,310.5478,0.6694640,9.250133,9.503690
8,2,arima,5.975336790,452.6574,400.5629,0.6850737,9.433383,9.721065
14,3,arimah,7.146360954,468.1836,408.3774,0.7087413,9.751145,10.064849
11,4,prophet,4.014240821,632.3472,404.8437,0.9336337,12.738781,12.325154
15,5,etsb,10.484768232,613.8696,451.9989,0.9358517,12.764499,12.728196
12,6,thetah,0.043348948,618.0366,467.8389,0.9602068,13.090917,13.315932
13,7,etsh,0.236940861,618.1173,468.0000,0.9606048,13.096381,13.320389
6,8,ets,0.063420137,618.3435,468.0000,0.9614541,13.107456,13.347997
5,9,theta,0.016820113,618.7948,467.8391,0.9628165,13.125469,13.378067


In [48]:
ts_obj <- ts(store_sales_aggregated[[8]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output20 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output20

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,20.245966514,359.7700,272.5546,0.7146145,8.006906,8.004206
14,2,arimah,64.663600604,374.9490,276.0433,0.7228356,8.069515,8.248563
8,3,arima,5.767160416,391.6864,288.4716,0.7751904,8.643831,8.580627
11,4,prophet,4.105767886,443.3011,317.5654,0.8830240,9.794508,9.849722
6,5,ets,0.135107835,445.7296,331.3902,0.8895388,9.865164,9.971739
15,6,etsb,14.062864542,451.8214,313.5231,0.8907348,9.880421,9.807323
13,7,etsh,0.289467653,446.0056,332.9883,0.8902983,9.873564,9.975458
3,8,croston,50.527187824,447.7865,356.3999,0.8951405,9.925701,10.077828
9,9,arimad,0.966522137,488.3965,319.4172,0.9414050,10.432368,10.455659


In [49]:
ts_obj <- ts(store_sales_aggregated[[9]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output21 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output21

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8,1,arima,6.953328133,711.4329,499.4383,1.114245,14.26023,14.13780
14,2,arimah,22.809327682,713.2544,484.5375,1.116213,14.26970,14.15514
9,3,arimad,0.412194411,872.7055,639.3850,1.405748,17.92275,18.01204
4,4,holtw,0.043982108,987.2509,511.7285,1.527459,19.69430,20.35644
15,5,etsb,7.220011234,1004.8986,699.5858,1.649533,21.39002,20.79312
2,6,snaive,0.001273553,1097.8496,599.5000,1.617436,21.83368,19.68042
6,7,ets,0.072550615,999.5730,776.3331,1.681732,21.82686,21.63819
11,8,prophet,3.858445644,1008.2290,727.9843,1.656994,21.48928,20.94811
13,9,etsh,0.232513030,999.6360,780.8000,1.685731,21.88008,21.72312


In [50]:
ts_obj <- ts(store_sales_aggregated[[10]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output22 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output22

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,holtw,0.053398450,887.8528,636.4024,1.399806,13.99852,13.57936
9,2,arimad,0.723295053,954.1717,631.0966,1.468180,14.62391,13.77041
7,3,etsd,0.077438911,935.9426,685.0630,1.456620,14.51077,13.99494
14,4,arimah,61.201413949,1004.1709,656.5180,1.573727,15.52438,15.22072
1,5,strend,0.090404431,1030.7861,728.1318,1.572943,15.62976,14.50131
3,6,croston,50.759570360,1084.4943,713.0000,1.705053,16.81942,16.70413
12,7,thetah,0.037305435,1192.8300,807.4080,1.865516,18.36077,18.15904
13,8,etsh,0.205828587,1197.8738,799.5477,1.873239,18.43497,18.22044
11,9,prophet,3.953573545,1237.5337,800.2481,1.934723,19.33540,17.34413


In [20]:
ts_obj <- ts(store_sales_aggregated[[11]],
               start = as.numeric(format(store_sales_aggregated$date[1], "%Y")),
               frequency = 365)
output23 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output23

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,34.475233952,665.4678,535.7372,1.026125,13.99380,13.91498
9,2,arimad,0.452206691,686.5924,482.8318,1.058049,14.36356,14.03774
4,3,holtw,0.048491796,715.6433,434.2379,1.083136,14.67184,14.52463
8,4,arima,8.946989457,761.5528,537.6716,1.168348,15.90973,15.76716
3,5,croston,58.087607066,806.9791,634.2060,1.235789,16.84446,17.12904
11,6,prophet,5.713483334,812.2369,597.9351,1.254109,17.13973,16.62790
15,7,etsb,10.462328831,876.6126,596.3760,1.313761,17.93638,17.60278
13,8,etsh,0.248725573,885.4559,694.4857,1.377909,18.55727,19.85995
10,9,tbats,8.359361490,906.8491,657.6356,1.395872,19.30553,18.40151


In [21]:
ts_obj <- ts(state_category_sales[[2]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output24 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output24

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,1.567962e+01,1212.433,817.0286,0.7570243,8.258685,8.407635
9,2,arimad,4.376192e-01,1474.191,906.2267,0.8989142,9.747185,9.887961
14,3,arimah,1.698336e+02,1452.542,997.8108,0.9090367,9.965114,9.992352
8,4,arima,5.938511e+00,1445.723,1118.4568,0.9232730,10.133338,10.217182
11,5,prophet,4.377265e+00,2190.506,1061.1168,1.2283618,13.490331,12.447773
15,6,etsb,1.275163e+01,2113.058,1216.3566,1.2264475,13.471769,12.730057
4,7,holtw,5.293393e-02,2088.613,1497.2069,1.2763889,13.854701,14.529875
3,8,croston,5.446311e+01,2068.110,1777.0060,1.3441220,14.825242,14.860200
1,9,strend,9.335160e-02,2554.593,1754.2947,1.5603622,17.392657,16.338451


In [22]:
ts_obj <- ts(state_category_sales[[3]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output25 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output25

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,10.447226922,244.3827,171.9526,0.8676493,10.35386,10.35813
14,2,arimah,20.884633859,244.4022,184.4838,0.8767176,10.45772,10.47593
8,3,arima,5.898804903,252.3020,165.6949,0.8893206,10.60740,10.52505
13,4,etsh,0.232740641,269.2083,215.7955,1.0070776,12.02562,12.09943
6,5,ets,0.053631385,270.0845,218.3870,1.0094774,12.05289,12.12473
12,6,thetah,0.043015957,269.9708,221.5273,1.0158220,12.12878,12.23820
5,7,theta,0.011526823,270.7459,220.7273,1.0166958,12.13772,12.24188
3,8,croston,54.476198753,274.4064,218.1951,1.0254748,12.23688,12.32192
4,9,holtw,0.053189754,305.7320,246.0942,1.1195938,13.21422,13.54835


In [23]:
ts_obj <- ts(state_category_sales[[4]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output26 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output26

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
8,1,arima,6.329835574,529.2639,301.0969,0.7406309,8.201967,7.977882
10,2,tbats,9.935359399,502.8265,340.3921,0.7482078,8.383304,8.329597
14,3,arimah,16.611949762,530.1266,318.1515,0.7623959,8.465217,8.295649
9,4,arimad,0.456600428,555.6088,327.5313,0.7958629,8.899091,8.783028
4,5,holtw,0.069636663,915.5087,515.9903,1.4072968,15.790065,15.951439
11,6,prophet,4.566168865,917.2173,721.3236,1.4759455,17.067856,16.745663
2,7,snaive,0.001597087,1001.0461,628.5000,1.4851022,17.330520,15.922328
3,8,croston,53.670734326,921.6973,739.5000,1.5107638,17.478576,17.342861
15,9,etsb,7.035704056,920.6693,763.4709,1.5236635,17.632912,17.588679


In [24]:
ts_obj <- ts(state_category_sales[[5]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output27 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output27

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,18.809263865,841.5688,595.1530,0.6921074,9.421624,9.313858
14,2,arimah,43.635417620,904.4188,677.5163,0.7629778,10.244088,10.399099
8,3,arima,5.785705566,933.7837,675.5320,0.7854082,10.575608,10.613146
15,4,etsb,13.757311821,1147.2838,637.4073,0.8929172,12.001314,11.373252
13,5,etsh,0.325283051,1070.4995,688.0311,0.9079327,12.194001,12.097194
11,6,prophet,5.364008904,1179.0121,718.2413,0.9036036,12.153389,11.302641
6,7,ets,0.144083420,1071.3910,725.1698,0.9196952,12.349945,12.346481
9,8,arimad,0.868907770,1123.8374,790.1280,0.9281391,12.377643,12.483009
3,9,croston,55.193735123,1085.9388,745.5714,0.9426664,12.649644,12.791603


In [25]:
ts_obj <- ts(state_category_sales[[6]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output28 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output28

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,16.247560024,150.0164,93.34867,0.8516281,10.82533,10.50225
8,2,arima,6.026966095,154.4835,95.94267,0.8805054,11.22789,10.81163
10,3,tbats,9.435608705,152.0757,91.21058,0.8929239,11.34534,11.17661
12,4,thetah,0.054799318,156.8167,105.53224,0.9215862,11.71434,11.56840
13,5,etsh,0.273639123,157.7285,105.40873,0.9230608,11.73243,11.55492
5,6,theta,0.026719888,156.9914,105.94302,0.9226573,11.72781,11.58498
6,7,ets,0.100403627,158.4583,109.00412,0.9266825,11.77753,11.60645
15,8,etsb,9.497709990,156.7599,118.37802,0.9429491,11.98296,12.10910
3,9,croston,54.546482563,159.6352,110.04392,0.9315541,11.84275,11.63230


In [26]:
ts_obj <- ts(state_category_sales[[7]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output29 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output29

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,1,tbats,9.785896142,292.1409,209.5625,0.7350664,8.269414,8.227229
8,2,arima,6.902509133,331.5887,260.0274,0.8280889,9.313156,9.469820
14,3,arimah,15.389491479,326.1255,260.4275,0.8407240,9.468391,9.694092
9,4,arimad,0.476320823,376.5418,330.6870,0.9956647,11.130906,11.401441
11,5,prophet,4.625044346,422.2040,328.6356,1.0759990,12.061348,12.290663
6,6,ets,0.071608226,426.7640,360.2550,1.1324637,12.679471,13.035272
12,7,thetah,0.044645389,428.2938,358.4943,1.1374256,12.733395,13.110403
13,8,etsh,0.253997644,429.2611,355.3925,1.1406433,12.768381,13.159403
3,9,croston,54.115574280,433.2796,354.4127,1.1646930,13.021126,13.459798


In [27]:
ts_obj <- ts(state_category_sales[[8]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output30 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output30

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,holtw,6.424228e-02,1749.705,1182.526,1.322260,14.99348,14.99223
10,2,tbats,1.462191e+01,1777.026,1286.500,1.341477,15.38843,15.10646
14,3,arimah,3.383462e+02,1764.156,1248.101,1.364312,15.52861,15.25171
9,4,arimad,9.254231e-01,1990.062,1407.022,1.471185,16.79117,16.05700
7,5,etsd,1.096217e-01,2007.072,1400.493,1.525107,17.40972,16.79158
3,6,croston,6.224575e+01,2000.045,1406.500,1.527960,17.36649,17.40110
1,7,strend,9.417264e-02,2291.690,1197.666,1.613917,18.58010,16.56021
8,8,arima,6.985427e+00,2073.121,1411.880,1.577758,17.95145,17.50017
11,9,prophet,4.791143e+00,2208.433,1529.937,1.651710,18.90896,17.28633


In [28]:
ts_obj <- ts(state_category_sales[[9]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output31 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output31

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,33.14784273,188.4524,133.0908,0.9645888,15.74855,15.58755
10,2,tbats,13.36851796,186.9208,136.1814,0.9662261,15.79751,15.77288
8,3,arima,19.64022231,190.7380,143.2972,0.9773756,15.96519,15.78577
13,4,etsh,0.23601627,214.5110,143.0999,1.1181526,18.28730,17.99520
11,5,prophet,4.16831477,215.0728,148.1543,1.1141961,18.21821,18.10131
6,6,ets,0.05729342,214.5119,144.2322,1.1199092,18.31653,18.05269
9,7,arimad,1.29302088,217.9026,156.1199,1.1194611,18.23246,18.28350
15,8,etsb,10.89609313,213.7100,163.0294,1.1384556,18.62223,18.75738
4,9,holtw,0.07224035,240.4541,139.6644,1.1824305,19.23777,18.41400


In [29]:
ts_obj <- ts(state_category_sales[[10]],
               start = as.numeric(format(state_category_sales$date[1], "%Y")),
               frequency = 365)
output32 <- evaluateModels(ts_obj, steps.ahead, model.names, windows.num)
output32

Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in TSrepr::mdae(realVals, foreVals):
“subscript out of bounds (index 84 >= vector size 84)”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() if you need seasonal forecasts.”
Warning message in forecast::ets(xx):
“I can't handle data with frequency greater than 24. Seasonality will be ignored. Try stlf() i

,rank,model,elapsed,RMSE,MDAE,MASE,SMAPE,MAAPE
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,1,arimah,8.075101614,542.8618,382.2852,1.343783,16.23211,16.47439
9,2,arimad,2.086598476,538.8672,430.2086,1.352523,16.23301,16.26738
10,3,tbats,8.413754145,557.8606,381.5036,1.343434,16.28494,16.56106
4,4,holtw,0.049139023,588.3444,377.1731,1.413512,17.06890,16.61920
8,5,arima,6.858892282,556.0394,392.5965,1.369197,16.58038,16.70415
11,6,prophet,4.281549613,620.7811,468.2693,1.584555,19.12559,18.96493
13,7,etsh,0.215281645,633.6703,504.9686,1.672188,20.14524,20.77186
12,8,thetah,0.046859980,635.8667,516.9884,1.680318,20.23761,20.93698
2,9,snaive,0.001425346,710.0357,465.5000,1.638153,20.63632,17.99290
